In [1]:
/* 22-15 */
USE master;
GO
CREATE RESOURCE POOL priority_app_queries WITH ( MIN_CPU_PERCENT = 25,
MAX_CPU_PERCENT = 75,
MIN_MEMORY_PERCENT = 25,
MAX_MEMORY_PERCENT = 75);
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.004

In [2]:
/* second resource group */
USE master;
GO
CREATE RESOURCE POOL ad_hoc_queries WITH ( MIN_CPU_PERCENT = 5,
MAX_CPU_PERCENT = 25,
MIN_MEMORY_PERCENT = 5,
MAX_MEMORY_PERCENT = 25,
MAX_IOPS_PER_VOLUME = 50);
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.008

In [3]:
/* alter the resource pool */
USE master;
GO
ALTER RESOURCE POOL ad_hoc_queries 
WITH ( MIN_MEMORY_PERCENT = 10, MAX_MEMORY_PERCENT = 50, MAX_IOPS_PER_VOLUME = 75);
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.010

In [4]:
USE master;
GO
SELECT pool_id,name AS PoolName
,min_cpu_percent,max_cpu_percent
,min_memory_percent,max_memory_percent, max_iops_per_volume
FROM sys.resource_governor_resource_pools rp
WHERE rp.pool_id > 2;
GO

Commands completed successfully.

(2 rows affected)

Total execution time: 00:00:00.042

pool_id,PoolName,min_cpu_percent,max_cpu_percent,min_memory_percent,max_memory_percent,max_iops_per_volume
256,priority_app_queries,25,75,25,75,0
257,ad_hoc_queries,5,25,10,50,75


In [5]:
/* create some workload groups now */
USE master;
GO
CREATE WORKLOAD GROUP application_alpha 
WITH ( IMPORTANCE = HIGH,
REQUEST_MAX_MEMORY_GRANT_PERCENT = 75,
REQUEST_MAX_CPU_TIME_SEC = 75,
REQUEST_MEMORY_GRANT_TIMEOUT_SEC = 120,
MAX_DOP = 8,
GROUP_MAX_REQUESTS = 8 ) 
USING priority_app_queries;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.013

In [6]:
/* second workload group */
USE master;
GO
CREATE WORKLOAD GROUP application_beta 
WITH ( IMPORTANCE = LOW,
REQUEST_MAX_MEMORY_GRANT_PERCENT = 50,
REQUEST_MAX_CPU_TIME_SEC = 50,
REQUEST_MEMORY_GRANT_TIMEOUT_SEC = 360,
MAX_DOP = 1,
GROUP_MAX_REQUESTS = 4 ) 
USING priority_app_queries;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.010

In [7]:
/* Alter a workload */
USE master;
GO
ALTER WORKLOAD GROUP application_beta WITH ( IMPORTANCE = MEDIUM);
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.004

In [8]:
/* workload group for the adhoc pool */
USE master;
GO
CREATE WORKLOAD GROUP adhoc_users 
WITH ( IMPORTANCE = LOW,
REQUEST_MAX_MEMORY_GRANT_PERCENT = 100,
REQUEST_MAX_CPU_TIME_SEC = 120,
REQUEST_MEMORY_GRANT_TIMEOUT_SEC = 360,
MAX_DOP = 1,
GROUP_MAX_REQUESTS = 5 ) 
USING ad_hoc_queries;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.011

In [9]:
/* confirm the setup */
USE master;
GO
SELECT name AS GrpName,
Importance AS impt,
request_max_memory_grant_percent AS max_m_g,
request_max_cpu_time_sec AS max_cpu_sec,
request_memory_grant_timeout_sec AS m_g_to,
max_dop,
group_max_requests AS max_req,
pool_id
FROM sys.resource_governor_workload_groups
WHERE pool_id > 2;

Commands completed successfully.

(3 rows affected)

Total execution time: 00:00:00.064

GrpName,impt,max_m_g,max_cpu_sec,m_g_to,max_dop,max_req,pool_id
application_alpha,High,75,75,120,8,8,256
application_beta,Medium,50,50,360,1,4,256
adhoc_users,Low,100,120,360,1,5,257


In [10]:
/* classifier function */
USE master;
GO
CREATE FUNCTION dbo.RECIPES_classifier()
RETURNS sysname
WITH SCHEMABINDING
AS
BEGIN
DECLARE @resource_group_name sysname;
IF SUSER_SNAME() IN ('AppLoginl', 'AppLogin2') 
    SET @resource_group_name = 'application_alpha';
IF SUSER_SNAME() IN ('AppLogin3', 'AppLogin4') 
    SET @resource_group_name = 'application_beta';
IF HOST_NAME() IN ('Workstationl234', 'Workstation4235') 
    SET @resource_group_name = 'adhoc_users';
-- If the resource group is still unassigned, use default 
IF @resource_group_name IS NULL 
    SET @resource_group_name = 'default';
RETURN @resource_group_name;
END
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.012

In [11]:
/* activate the classifier function */
USE master;
GO
-- Assign the classifier function
ALTER RESOURCE GOVERNOR
WITH (CLASSIFIER_FUNCTION = dbo.RECIPES_classifier);
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.011

In [12]:
/* enable the RG configuration */
USE master;
GO
ALTER RESOURCE GOVERNOR RECONFIGURE;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.039

In [13]:
/* confirm the setup is complete */
USE master;
GO
SELECT OBJECT_NAME(classifier_function_id,DB_ID('master')) FuncName,
is_enabled 
FROM sys.resource_governor_configuration;

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.048

FuncName,is_enabled
RECIPES_classifier,1


In [14]:
/* disable the settings */
USE master;
GO
ALTER RESOURCE GOVERNOR DISABLE;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.009

In [15]:
/* remove the workgroups and pools */
USE master;
GO
DROP WORKLOAD GROUP application_alpha;
DROP WORKLOAD GROUP application_beta;
DROP WORKLOAD GROUP adhoc_users;
DROP RESOURCE POOL ad_hoc_queries;
DROP RESOURCE POOL priority_app_queries;

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.022

In [16]:
/* drop the classifier function */
USE master;
GO
ALTER RESOURCE GOVERNOR
WITH (CLASSIFIER_FUNCTION = NULL);
DROP FUNCTION dbo.RECIPES_classifier;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.012